In [1]:
import sqlite3
import pandas as pd

# Percorso al tuo file .sqlite
db_path = "D:/files/Bankit.sqlite"
pd.set_option('display.max_rows', None)

In [2]:
valore_cercato = 'deterioramento'

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Otteniamo tutte le tabelle
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tabelle = [row[0] for row in cursor.fetchall()]

trovate = []

for tabella in tabelle:
    try:
        cursor.execute(f"PRAGMA table_info('{tabella}')")
        colonne = [r[1] for r in cursor.fetchall()]
        
        # Escludiamo 'DATA_OSS' e 'VALORE'
        colonne = [col for col in colonne if col.upper() not in ('DATA_OSS', 'VALORE', 'STATUS')]
        
        for col in colonne:
            try:
                query = f'SELECT 1 FROM "{tabella}" WHERE "{col}" = ? LIMIT 1'
                cursor.execute(query, (valore_cercato,))
                if cursor.fetchone():
                    trovate.append((tabella, col))
                    break  # salta alla prossima tabella
            except sqlite3.OperationalError:
                continue
    except Exception as e:
        print(f"Errore con la tabella {tabella}: {e}")

# Mapping tabella -> descrizione, usando solo la parte prima dell'underscore
descrizioni = {}
try:
    cursor.execute("SELECT tabella, descrizione FROM tabelle")
    for tabella_completa, descrizione in cursor.fetchall():
        base_tabella = tabella_completa.split('_')[0].upper()
        if base_tabella not in descrizioni:
            descrizioni[base_tabella] = (tabella_completa, descrizione)
except Exception as e:
    print(f"Errore leggendo la tabella 'tabelle': {e}")

conn.close()

# Stampiamo i risultati con descrizione
for tabella, colonna in sorted(trovate):
    match = descrizioni.get(tabella.upper())
    if match:
        tabella_completa, desc = match
        print(f"✔ Trovato in: '{tabella}' ({desc}, da '{tabella_completa}'), colonna '{colonna}'")
    else:
        print(f"✔ Trovato in: '{tabella}' ((descrizione non trovata)), colonna '{colonna}'")


Errore con la tabella TRI30633: database disk image is malformed


In [3]:
valore_cercato = 'deterioramento'

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Otteniamo tutte le tabelle
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tabelle = [row[0] for row in cursor.fetchall()]

trovate = []

for tabella in tabelle:
    try:
        cursor.execute(f"PRAGMA table_info('{tabella}')")
        colonne = [r[1] for r in cursor.fetchall()]
        
        for col in colonne:
            try:
                query = f'SELECT 1 FROM "{tabella}" WHERE "{col}" = ? LIMIT 1'
                cursor.execute(query, (valore_cercato,))
                if cursor.fetchone():
                    trovate.append((tabella, col))
                    break  # salta alla prossima tabella
            except sqlite3.OperationalError:
                continue
    except Exception as e:
        print(f"Errore con la tabella {tabella}: {e}")

# Mapping tabella -> descrizione, usando solo la parte prima dell'underscore
descrizioni = {}
try:
    cursor.execute("SELECT tabella, descrizione FROM tabelle")
    for tabella_completa, descrizione in cursor.fetchall():
        base_tabella = tabella_completa.split('_')[0].upper()
        if base_tabella not in descrizioni:
            descrizioni[base_tabella] = (tabella_completa, descrizione)
except Exception as e:
    print(f"Errore leggendo la tabella 'tabelle': {e}")


conn.close()

# Stampiamo i risultati con descrizione
for tabella, colonna in sorted(trovate):
    match = descrizioni.get(tabella.upper())
    if match:
        tabella_completa, desc = match
        print(f"✔ Trovato in: '{tabella}' ({desc}, da '{tabella_completa}'), colonna '{colonna}'")
    else:
        print(f"✔ Trovato in: '{tabella}' ((descrizione non trovata)), colonna '{colonna}'")



Errore con la tabella TRI30633: database disk image is malformed


In [3]:
import sqlite3
import pandas as pd

valori_richiesti = ['S11', 'S14BI4']

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Ottieni tutte le tabelle
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tabelle = [row[0] for row in cursor.fetchall()]

risultati = []
for tabella in tabelle:
    riga = {'tabella': tabella}
    try:
        cursor.execute(f'PRAGMA table_info("{tabella}")')
        colonne = [r[1] for r in cursor.fetchall()]

        for valore in valori_richiesti:
            trovato = None
            for col in colonne:
                try:
                    cursor.execute(f'SELECT 1 FROM "{tabella}" WHERE "{col}" = ? LIMIT 1', (valore,))
                    if cursor.fetchone():
                        trovato = col
                        break
                except sqlite3.OperationalError:
                    continue
            riga[valore] = trovato
        risultati.append(riga)
    except Exception as e:
        print(f"Errore con la tabella {tabella}: {e}")

conn.close()

# Converte in DataFrame
df = pd.DataFrame(risultati)

# Conta solo i valori trovati nella colonna 'SET_CTP'
df["conteggio"] = df[valori_richiesti].apply(lambda row: sum(v == "SET_CTP" for v in row), axis=1)

# Filtra solo tabelle con almeno un valore trovato in SET_CTP
df = df[df["conteggio"] > 0]

# Ricarica le descrizioni dalla tabella 'tabelle'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

mappa_descrizioni = {}
try:
    cursor.execute("SELECT tabella, descrizione FROM tabelle")
    for tabella_completa, descrizione in cursor.fetchall():
        base = tabella_completa.split("_")[0].upper()
        if base not in mappa_descrizioni:
            mappa_descrizioni[base] = (tabella_completa, descrizione)
except Exception as e:
    print(f"Errore leggendo la tabella 'tabelle': {e}")

conn.close()

# Aggiunge la descrizione basandosi sulla parte base della tabella
df["descrizione"] = df["tabella"].apply(lambda x: mappa_descrizioni.get(x.upper(), ("", "(descrizione non trovata)"))[1])

# Riordina le colonne
col_riordinate = ["tabella", "descrizione"] + valori_richiesti + ["conteggio"]
df = df[col_riordinate]

# Ordina per conteggio decrescente
df = df.sort_values(by="conteggio", ascending=False)

# Visualizza
print(df.to_string(index=False))
# df.to_csv("valori_solo_in_SET_CTP.csv", index=False)


Errore con la tabella TRI30633: database disk image is malformed
 tabella                                                                                                                     descrizione     S11  S14BI4  conteggio
TDB30156                                                                                                       (descrizione non trovata) SET_CTP SET_CTP          2
TRI30603                                             Tasso di deterioramento annuale dei prestiti - default rettificato: numero affidati SET_CTP SET_CTP          2
TDC30021                                                                                                       (descrizione non trovata) SET_CTP SET_CTP          2
TRI30031                                          Sofferenze (al lordo delle svalutazioni e al netto dei passaggi a perdita): utilizzato SET_CTP SET_CTP          2
TRI30033                                                   Sofferenze (al lordo delle svalutazioni e dei passaggi a

In [4]:
tabella = 'TFR10420'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Corretto uso di PRAGMA
cursor.execute(f"PRAGMA table_info({tabella})")
tutte_le_colonne = [row[1] for row in cursor.fetchall()]

# Colonne da esplorare (escluse quelle tecniche)
colonne_da_esplorare = [col for col in tutte_le_colonne if col not in ("DATA_OSS", "VALORE", "STATUS")]

# Raccolta dei valori unici
risultati = {}
for col in colonne_da_esplorare:
    try:
        cursor.execute(f'SELECT DISTINCT "{col}" FROM {tabella} ORDER BY 1')
        risultati[col] = [row[0] if row[0] is not None else "" for row in cursor.fetchall()]
    except Exception as e:
        print(f"Errore nella colonna {col}: {e}")

conn.close()

# Conversione in DataFrame e sostituzione dei NaN con stringa vuota
df_unici = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in risultati.items()])).fillna("")

# Opzioni display per Pandas
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

print("\nValori unici per ciascuna colonna (escluse DATA_OSS e VALORE):\n")
print(df_unici)


Valori unici per ciascuna colonna (escluse DATA_OSS e VALORE):

    DESINV DURORI ENTE_SEGN    FENEC  LOC_CTP RESIDENZA1 TIPTASSO
0     10.0   18.0   1070001  1001530  9999999         IT     80.0
1     40.0          1100010                IT               800.0
2     70.0                                ITC             10000.0
3     80.0                               ITC1                    
4    110.0                              ITC11                    
5    120.0                              ITC12                    
6    902.0                              ITC13                    
7    905.0                              ITC14                    
8    909.0                              ITC15                    
9    912.0                              ITC16                    
10   914.0                              ITC17                    
11   916.0                              ITC18                    
12   997.0                               ITC2                    
13   998.0 

In [3]:
tabella = 'TFR10420'
domini = 'STACORIS'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
mappa_domini = {
    "ATECO_CTP": "ATECO",
    "SEDELEG_SOGG":"TERRITORIO",'LOC_CTP':"TERRITORIO",
    'DIVISA1':'VALUTISO',
    "CLASSE_ACCORD": "CLAGRAND",
    "LIVAFF":"PLURAFF",
    'DURORI':'DURATA',
    "ENTE_SEGN":"AZIENDA",
    "FENEC":"FENOMECON",
    "SET_CTP":"SETTORIST",
    'DESINV':'DESINV'
    # altri domini verranno aggiunti se identificati
}

# Elenco colonne da esplorare
cursor.execute(f"PRAGMA table_info({tabella})")
tutte_le_colonne = [row[1] for row in cursor.fetchall()]
colonne_da_esplorare = [col for col in tutte_le_colonne if col not in ("DATA_OSS", "VALORE", "STATUS")]

righe_uniche = []

# Estrazione valori + descrizioni
for col in colonne_da_esplorare:
    dominio = mappa_domini.get(col)
    if dominio is None:
        print(f"⚠️ Nessun dominio disponibile per la colonna '{col}', verrà saltata.")
        continue

    try:
        query = f'''
        SELECT DISTINCT '{col}' AS colonna,
                        CAST(t."{col}" AS TEXT) AS codice,
                        COALESCE(d.Descrizione, '') AS descrizione
        FROM {tabella} t

        LEFT JOIN "DOMAIN-STAFINRA-MULTICUBE" d

            ON CAST(t."{col}" AS TEXT) = CAST(d.Elemento AS TEXT)
           AND d.Dominio = '{dominio}'
        WHERE t."{col}" IS NOT NULL
        ORDER BY codice
        '''
        cursor.execute(query)
        righe_uniche.extend(cursor.fetchall())
    except Exception as e:
        print(f"Errore nella colonna {col}: {e}")

conn.close()

pd.DataFrame(righe_uniche, columns=["Colonna", "Codice", "Descrizione"])


⚠️ Nessun dominio disponibile per la colonna 'RESIDENZA1', verrà saltata.
⚠️ Nessun dominio disponibile per la colonna 'TIPTASSO', verrà saltata.


,Colonna,Codice,Descrizione
0,DESINV,10,Investimenti non finanziari: costruzioni - abi...
1,DESINV,110,Altri investimenti: acquisto di beni durevoli ...
2,DESINV,120,Investimenti finanziari
3,DESINV,40,Investimenti non finanziari: costruzioni - ope...
4,DESINV,70,Investimenti: acquisto immobili - abitazioni f...
5,DESINV,80,Investimenti: acquisto immobili - abitazioni d...
6,DESINV,902,Investimenti non finanziari: costruzioni - fab...
7,DESINV,905,"Investimenti non finanziari: macchine, attrezz..."
8,DESINV,909,Investimenti: acquisto di immobili - altri imm...
9,DESINV,912,Investimenti non finanziari: investimenti in c...
